In [5]:
from pyspark.sql import SparkSession, functions as fn
from pyspark.sql.types import *
from pyspark.sql.functions import avg, sum, min, mean, max, count, expr
from pyspark.sql.window import Window

In [6]:
spark = SparkSession.builder \
.master("local") \
.appName("aggregate_df3") \
.getOrCreate()

In [7]:
df1 = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.option("path", "D:\\BigData\\Week12- Apache Spark - Structured API Part-2\\windowdata.csv") \
.load()

In [10]:
myWindow = Window.partitionBy("country") \
.orderBy("weeknum") \
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [11]:
df2 = df1.withColumn("runningTotal", sum("invoicevalue").over(myWindow))
df2.show()

+-------+-------+-----------+-------------+------------+------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|      runningTotal|
+-------+-------+-----------+-------------+------------+------------------+
| Sweden|     50|          3|         3714|      2646.3|            2646.3|
|Germany|     48|         11|         1795|     3309.75|           3309.75|
|Germany|     49|         12|         1852|     4521.39|           7831.14|
|Germany|     50|         15|         1973|     5065.79|          12896.93|
|Germany|     51|          5|         1103|     1665.91|          14562.84|
| France|     48|          4|         1299|     2808.16|           2808.16|
| France|     49|          9|         2303|     4527.01|           7335.17|
| France|     50|          6|          529|      537.32|           7872.49|
| France|     51|          5|          847|     1702.87|           9575.36|
|Belgium|     48|          1|          528|       346.1|             346.1|
|Belgium|   

In [12]:
df1.createOrReplaceTempView("windowData")
spark.sql("""select country, weeknum, numinvoices, totalquantity, invoicevalue, 
sum(invoicevalue) over (partition by country order by weeknum
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as runningTotal from windowData""").show()

+-------+-------+-----------+-------------+------------+------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|      runningTotal|
+-------+-------+-----------+-------------+------------+------------------+
| Sweden|     50|          3|         3714|      2646.3|            2646.3|
|Germany|     48|         11|         1795|     3309.75|           3309.75|
|Germany|     49|         12|         1852|     4521.39|           7831.14|
|Germany|     50|         15|         1973|     5065.79|          12896.93|
|Germany|     51|          5|         1103|     1665.91|          14562.84|
| France|     48|          4|         1299|     2808.16|           2808.16|
| France|     49|          9|         2303|     4527.01|           7335.17|
| France|     50|          6|          529|      537.32|           7872.49|
| France|     51|          5|          847|     1702.87|           9575.36|
|Belgium|     48|          1|          528|       346.1|             346.1|
|Belgium|   